# RFM Analysis

### 1. Data Preparation

In [1]:
# load library
import pandas as pd

In [2]:
# load the dataset
data = 'superstore_dataset2011-2015.csv'
dt = pd.read_csv(data, encoding='cp1252')
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Row ID          51290 non-null  int64  
 1   Order ID        51290 non-null  object 
 2   Order Date      51290 non-null  object 
 3   Ship Date       51290 non-null  object 
 4   Ship Mode       51290 non-null  object 
 5   Customer ID     51290 non-null  object 
 6   Customer Name   51290 non-null  object 
 7   Segment         51290 non-null  object 
 8   City            51290 non-null  object 
 9   State           51290 non-null  object 
 10  Country         51290 non-null  object 
 11  Postal Code     9994 non-null   float64
 12  Market          51290 non-null  object 
 13  Region          51290 non-null  object 
 14  Product ID      51290 non-null  object 
 15  Category        51290 non-null  object 
 16  Sub-Category    51290 non-null  object 
 17  Product Name    51290 non-null 

The dataset contain 24 columns with no null values in each column. For the ease of coding, the name of the dataset columns should be written in lower case letter and has no space between each coloumn name words.

In [3]:
# clean column names
dt.columns = [col.lower().replace(' ', '_') for col in dt.columns]

In [5]:
# select column for performing rfm analysis
df = dt[['order_id', 'order_date', 'customer_id', 'sales']]

# convert order_date column dtype to dataframe
df['order_date'] = pd.to_datetime(df['order_date'],dayfirst=False, format='mixed')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     51290 non-null  object        
 1   order_date   51290 non-null  datetime64[ns]
 2   customer_id  51290 non-null  object        
 3   sales        51290 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.6+ MB


C:\Users\ahard\AppData\Local\Temp\ipykernel_25632\1761538230.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['order_date'] = pd.to_datetime(df['order_date'],dayfirst=False, format='mixed')


order_date           
                    min        max
order_id                          
AE-2011-9160 2011-03-10 2011-03-10
AE-2013-1130 2013-10-14 2013-10-14
AE-2013-1530 2013-12-31 2013-12-31
AE-2014-2840 2014-05-11 2014-05-11
AE-2014-3830 2014-12-13 2014-12-13
...                 ...        ...
ZI-2014-7160 2014-10-30 2014-10-30
ZI-2014-7610 2014-03-24 2014-03-24
ZI-2014-9540 2014-09-06 2014-09-06
ZI-2014-9550 2014-12-18 2014-12-18
ZI-2014-9650 2014-05-21 2014-05-21

[25035 rows x 2 columns]